In [ ]:
import os
import requests
import zipfile
from tqdm import tqdm

def download_tiny_imagenet(url, path):
    if not os.path.exists(path):
        os.makedirs(path)
    zip_path = os.path.join(path, 'tiny-imagenet-200.zip')
    if not os.path.exists(zip_path):
        print("Downloading Tiny ImageNet...")
        r = requests.get(url, stream=True)
        with open(zip_path, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024)):
                if chunk:
                    f.write(chunk)
        print("Download complete.")

    print("Extracting Tiny ImageNet...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path)
    print("Extraction complete.")

# URL for the Tiny ImageNet dataset
url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
path = 'tiny-imagenet'

download_tiny_imagenet(url, path)

# The dataset is now in the 'tiny-imagenet/tiny-imagenet-200' directory

242286it [00:14, 16267.53it/s]


Download complete.
Extracting Tiny ImageNet...
Extraction complete.


In [ ]:
import torch
import torchvision.models as models

# Load a pretrained ResNet-18 model
model = models.resnet18(pretrained=True)

# Get the number of input features for the classifier
num_ftrs = model.fc.in_features

# Replace the final fully connected layer
# The new 'fc' layer will have 200 output units
model.fc = torch.nn.Linear(num_ftrs, 200)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 152MB/s]


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 142502009 (142502009_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
!nvidia-smi


Sat Nov  8 16:34:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import wandb

# --- 1. Initialize W&B ---
wandb.init(
    project="tiny-imagenet-resnet",
    config={
        "learning_rate": 0.001,
        "architecture": "ResNet18",
        "dataset": "Tiny ImageNet",
        "epochs": 10,
        "batch_size": 128
    }
)
config = wandb.config

# --- 2. Check GPU ---
assert torch.cuda.is_available(), "GPU not found! Enable GPU in Runtime > Change runtime type."
device = torch.device("cuda")

# --- 3. Data Preparation ---
data_dir = 'tiny-imagenet/tiny-imagenet-200'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]),
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val'])
}

dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x],
                                   batch_size=config.batch_size,
                                   shuffle=True,
                                   num_workers=2)
    for x in ['train', 'val']
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# --- 4. Model Setup ---
model = models.resnet18(weights="IMAGENET1K_V1")

# Allow ResNet to work with 64x64 images
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))

# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)

# Mixed precision scaler
scaler = torch.cuda.amp.GradScaler()

# --- 5. Training Loop ---
for epoch in range(config.epochs):
    print(f"\nEpoch {epoch+1}/{config.epochs}")
    print("-" * 30)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                with torch.cuda.amp.autocast():  # Mixed precision block
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

        print(f"{phase.upper()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

        wandb.log({f"{phase}_loss": epoch_loss,
                   f"{phase}_accuracy": epoch_acc,
                   "epoch": epoch})

print("\nTraining Complete!")

# --- 6. Save + Log to W&B ---
torch.save(model.state_dict(), "resnet18_tiny_imagenet_fast.pth")

artifact = wandb.Artifact("resnet18-tiny-imagenet-fast", type="model")
artifact.add_file("resnet18_tiny_imagenet_fast.pth")
wandb.log_artifact(artifact)

wandb.finish()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: 142502009 (142502009_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Epoch 1/10
------------------------------


/tmp/ipython-input-751739113.py:75: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipython-input-751739113.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision block


TRAIN Loss: 3.5947 | Acc: 0.2586
VAL Loss: 7.8418 | Acc: 0.0064

Epoch 2/10
------------------------------
TRAIN Loss: 2.2247 | Acc: 0.4700
VAL Loss: 8.9442 | Acc: 0.0048

Epoch 3/10
------------------------------
TRAIN Loss: 1.8718 | Acc: 0.5379
VAL Loss: 9.5631 | Acc: 0.0051

Epoch 4/10
------------------------------
TRAIN Loss: 1.6618 | Acc: 0.5840
VAL Loss: 9.9140 | Acc: 0.0050

Epoch 5/10
------------------------------
TRAIN Loss: 1.4931 | Acc: 0.6221
VAL Loss: 10.3083 | Acc: 0.0051

Epoch 6/10
------------------------------
TRAIN Loss: 1.3478 | Acc: 0.6556
VAL Loss: 10.4387 | Acc: 0.0060

Epoch 7/10
------------------------------
TRAIN Loss: 1.2153 | Acc: 0.6878
VAL Loss: 10.7189 | Acc: 0.0051

Epoch 8/10
------------------------------
TRAIN Loss: 1.1023 | Acc: 0.7164
VAL Loss: 10.8743 | Acc: 0.0057

Epoch 9/10
------------------------------
TRAIN Loss: 0.9943 | Acc: 0.7444
VAL Loss: 11.0006 | Acc: 0.0063

Epoch 10/10
------------------------------
TRAIN Loss: 0.8912 | Acc: 0.771

epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_accuracy,▁▄▅▅▆▆▇▇██
train_loss,█▄▄▃▃▂▂▂▁▁
val_accuracy,█▁▂▂▂▆▂▅█▆
val_loss,▁▃▅▅▆▆▇▇▇█
epoch,9
train_accuracy,0.77164
train_loss,0.89116
val_accuracy,0.006
val_loss,11.26075


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi, create_repo

# Set your Hugging Face username and the desired model name
hf_username = "chershilhyde"
model_name = "resnet18-tiny-imagenet"
repo_id = f"{hf_username}/{model_name}"

# Create the repository on the Hub
create_repo(repo_id, exist_ok=True)

# Upload the model file
api = HfApi()
api.upload_file(
    path_or_fileobj="resnet18_tiny_imagenet_fast.pth",
    path_in_repo="pytorch_model.bin", # A standard name for PyTorch models
    repo_id=repo_id,
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...18_tiny_imagenet_fast.pth:   1%|1         |  567kB / 45.2MB            

CommitInfo(commit_url='https://huggingface.co/chershilhyde/resnet18-tiny-imagenet/commit/e159c0d37b33631ab73e0e947e291d3c955d5eaf', commit_message='Upload pytorch_model.bin with huggingface_hub', commit_description='', oid='e159c0d37b33631ab73e0e947e291d3c955d5eaf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chershilhyde/resnet18-tiny-imagenet', endpoint='https://huggingface.co', repo_type='model', repo_id='chershilhyde/resnet18-tiny-imagenet'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import HfApi

# The repo_id should be the same as the one you just used
repo_id = "chershilhyde/resnet18-tiny-imagenet"

# The content for your model card
readme_content = """---
license: mit
---

# ResNet-18 for Tiny ImageNet

This is a ResNet-18 model fine-tuned on the Tiny ImageNet dataset.

## Model Description

This model is a `torchvision.models.resnet18` model with the final fully connected layer replaced to classify the 200 classes of the Tiny ImageNet dataset.

## Training Data

The model was trained on the Tiny ImageNet dataset, which consists of 200 classes, each with 500 training images and 50 validation images. All images are 64x64 colored images.

## Training Procedure

The model was trained for 10 epochs using the following hyperparameters:
- **Optimizer:** SGD with momentum=0.9
- **Learning Rate:** 0.001
- **Batch Size:** 128
- **Loss Function:** Cross-Entropy Loss

The training was logged using Weights & Biases. You can find the training metrics [here](link-to-your-wandb-project).

## How to Use

To use this model, you can load it using PyTorch:

```python
import torch
from torchvision import models

# Initialize the model architecture
model = models.resnet18(weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 200)

# Load the fine-tuned weights from the Hub
# Note: You can also download the 'pytorch_model.bin' file manually
from huggingface_hub import hf_hub_download
model_weights = hf_hub_download(repo_id="chershilhyde/resnet18-tiny-imagenet", filename="pytorch_model.bin")
model.load_state_dict(torch.load(model_weights))
model.eval()
```
"""

# Upload the README.md file
api = HfApi()
api.upload_file(
    path_or_fileobj=readme_content.encode('utf-8'),
    path_in_repo="README.md",
    repo_id=repo_id,
    repo_type="model",
)


CommitInfo(commit_url='https://huggingface.co/chershilhyde/resnet18-tiny-imagenet/commit/1048f7e8654b054cc8ab7c9e971cd65bcfd49e16', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='1048f7e8654b054cc8ab7c9e971cd65bcfd49e16', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chershilhyde/resnet18-tiny-imagenet', endpoint='https://huggingface.co', repo_type='model', repo_id='chershilhyde/resnet18-tiny-imagenet'), pr_revision=None, pr_num=None)


--- Evaluating on: brightness_drift ---


Drifted Accuracy: 0.0056
ALERT: Accuracy 0.0056 is BELOW the threshold of 0.3003!
W&B Alert sent.


drift_accuracy,0.0056
drift_type,brightness_drift



--- Evaluating on: noise_drift ---


Drifted Accuracy: 0.0046
ALERT: Accuracy 0.0046 is BELOW the threshold of 0.3003!
W&B Alert sent.


drift_accuracy,0.0046
drift_type,noise_drift
